In [1]:
import pandas

training_data = pandas.read_csv('train_data.csv',encoding='cp1252')
training_data

,Unnamed: 0,data,flag
0,0,court agrees to expedite n.f.l.'s appeal the d...,0
1,1,no tsunami but fifa's corruption storm rages o...,0
2,2,chung backs bin hammam in fifa presidential ra...,0
3,3,rory mcilroy hangs on to slim lead at augusta ...,0
4,4,"figure skating: south korean skater stumbles, ...",0
...,...,...,...
1428,1428,the score: the first computer musician why eve...,6
1429,1429,apple fans line up to buy first batch of ipad ...,6
1430,1430,"nintendo 3ds is fun, addictive and pricey the ...",6
1431,1431,intel unveils laptops that include tablet feat...,6


In [2]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer


#GET VECTOR COUNT
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(training_data.data)

#SAVE WORD VECTOR
pickle.dump(count_vect.vocabulary_, open("count_vector.pkl","wb"))

In [3]:
from sklearn.feature_extraction.text import TfidfTransformer

#TRANSFORM WORD VECTOR TO TF IDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#SAVE TF-IDF
pickle.dump(tfidf_transformer, open("tfidf.pkl","wb"))

In [4]:
# Multinomial Naive Bayes

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

#clf = MultinomialNB().fit(X_train_tfidf, training_data.flag)
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.25, random_state=42)
clf = MultinomialNB().fit(X_train, y_train)

#SAVE MODEL
pickle.dump(clf, open("nb_model.pkl", "wb"))

In [6]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

target_names = ["sport", "world", "us", "business", "health", "entertainment", "sci_tech"]

docs_new = "Messi joins other football team"
docs_new = [docs_new]

#LOAD MODEL
loaded_vec = CountVectorizer(vocabulary=pickle.load(open("count_vector.pkl", "rb")))
loaded_tfidf = pickle.load(open("tfidf.pkl","rb"))
loaded_model = pickle.load(open("nb_model.pkl","rb"))

X_new_counts = loaded_vec.transform(docs_new)
X_new_tfidf = loaded_tfidf.transform(X_new_counts)
predicted = loaded_model.predict(X_new_tfidf)

print(target_names[predicted[0]])

sport


In [7]:
predicted = loaded_model.predict(X_test)
result_bayes = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_bayes.to_csv('res_bayes.csv', sep = ',')

for predicted_item, result in zip(predicted, y_test):
    print(target_names[predicted_item], ' - ', target_names[result])



world  -  world
sport  -  world
business  -  business
world  -  world
business  -  business
sport  -  sport
sport  -  sport
sport  -  sport
sport  -  us
sport  -  sci_tech
business  -  business
sport  -  entertainment
business  -  business
sport  -  sport
world  -  world
world  -  world
sport  -  entertainment
sport  -  sport
world  -  world
business  -  us
world  -  world
world  -  world
sport  -  health
sport  -  sport
business  -  sci_tech
sport  -  sport
sport  -  entertainment
business  -  business
business  -  business
business  -  business
world  -  world
sport  -  sport
sport  -  entertainment
sport  -  business
sport  -  entertainment
world  -  world
world  -  sport
sport  -  sport
sport  -  us
sport  -  sport
sport  -  us
sport  -  entertainment
sport  -  us
world  -  world
business  -  business
business  -  business
sport  -  us
business  -  business
sport  -  health
sport  -  us
world  -  world
sport  -  sport
business  -  sci_tech
business  -  business
business  -  busines

In [8]:
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import f1_score

confusion_mat = confusion_matrix(y_test,predicted)
print(confusion_mat)
from sklearn.metrics import classification_report
target_names = ["sport", "world", "us", "business", "health", "entertainment", "sci_tech"]

print(classification_report(y_test, predicted, target_names=target_names))

[[86  1  0  0  0  0  0]
 [ 6 66  0  1  0  0  0]
 [29  7  3  2  0  0  0]
 [17  3  0 46  0  0  0]
 [18  2  1  2  0  0  0]
 [32  1  0  3  0  4  0]
 [17  4  0  8  0  0  0]]
               precision    recall  f1-score   support

        sport       0.42      0.99      0.59        87
        world       0.79      0.90      0.84        73
           us       0.75      0.07      0.13        41
     business       0.74      0.70      0.72        66
       health       0.00      0.00      0.00        23
entertainment       1.00      0.10      0.18        40
     sci_tech       0.00      0.00      0.00        29

     accuracy                           0.57       359
    macro avg       0.53      0.39      0.35       359
 weighted avg       0.59      0.57      0.48       359



C:\Users\Alankar.gupta\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
from sklearn.neural_network import MLPClassifier

clf_neural = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1,max_iter=10000)

X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.25, random_state=42)

clf_neural.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(15,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=10000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [10]:
pickle.dump(clf_neural, open("softmax.pkl", "wb"))

In [11]:
predicted = clf_neural.predict(X_test)
result_softmax = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_softmax.to_csv('res_softmax.csv', sep = ',')

for predicted_item, result in zip(predicted, y_test):
    print(target_names[predicted_item], ' - ', target_names[result])


world  -  world
sport  -  world
business  -  business
world  -  world
business  -  business
sport  -  sport
sport  -  sport
sport  -  sport
us  -  us
health  -  sci_tech
business  -  business
health  -  entertainment
business  -  business
sport  -  sport
world  -  world
us  -  world
health  -  entertainment
sport  -  sport
business  -  world
us  -  us
world  -  world
world  -  world
sci_tech  -  health
sport  -  sport
health  -  sci_tech
sport  -  sport
health  -  entertainment
business  -  business
us  -  business
business  -  business
world  -  world
sport  -  sport
entertainment  -  entertainment
health  -  business
sci_tech  -  entertainment
world  -  world
world  -  sport
sport  -  sport
sport  -  us
sport  -  sport
sport  -  us
entertainment  -  entertainment
health  -  us
world  -  world
business  -  business
health  -  business
sport  -  us
business  -  business
health  -  health
health  -  us
world  -  world
sport  -  sport
business  -  sci_tech
business  -  business
business 

In [12]:
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import f1_score

confusion_mat = confusion_matrix(y_test,predicted)
print(confusion_mat)
from sklearn.metrics import classification_report
target_names = ["sport", "world", "us", "business", "health", "entertainment", "sci_tech"]

print(classification_report(y_test, predicted, target_names=target_names))

[[81  2  1  0  2  0  1]
 [ 2 58  4  2  6  1  0]
 [ 8  1 19  2 10  0  1]
 [ 1  0  3 41 17  0  4]
 [ 2  1  1  0 18  0  1]
 [ 3  0  1  0 20 14  2]
 [ 3  0  0  6  7  0 13]]
               precision    recall  f1-score   support

        sport       0.81      0.93      0.87        87
        world       0.94      0.79      0.86        73
           us       0.66      0.46      0.54        41
     business       0.80      0.62      0.70        66
       health       0.23      0.78      0.35        23
entertainment       0.93      0.35      0.51        40
     sci_tech       0.59      0.45      0.51        29

     accuracy                           0.68       359
    macro avg       0.71      0.63      0.62       359
 weighted avg       0.78      0.68      0.70       359



In [13]:
from sklearn import svm
clf_svm = svm.LinearSVC()
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.25, random_state=42)
clf_svm.fit(X_train_tfidf, training_data.flag)
pickle.dump(clf_svm, open("svm.pkl", "wb"))

In [14]:
predicted = clf_svm.predict(X_test)
result_svm = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_svm.to_csv('res_svm.csv', sep = ',')
for predicted_item, result in zip(predicted, y_test):
    print(target_names[predicted_item], ' - ', target_names[result])

world  -  world
world  -  world
business  -  business
world  -  world
business  -  business
sport  -  sport
sport  -  sport
sport  -  sport
us  -  us
sci_tech  -  sci_tech
business  -  business
entertainment  -  entertainment
business  -  business
sport  -  sport
world  -  world
world  -  world
entertainment  -  entertainment
sport  -  sport
world  -  world
us  -  us
world  -  world
world  -  world
health  -  health
sport  -  sport
sci_tech  -  sci_tech
sport  -  sport
entertainment  -  entertainment
business  -  business
business  -  business
business  -  business
world  -  world
sport  -  sport
entertainment  -  entertainment
business  -  business
entertainment  -  entertainment
world  -  world
sport  -  sport
sport  -  sport
us  -  us
sport  -  sport
us  -  us
entertainment  -  entertainment
us  -  us
world  -  world
business  -  business
business  -  business
us  -  us
business  -  business
health  -  health
us  -  us
world  -  world
sport  -  sport
sci_tech  -  sci_tech
business  

In [15]:
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import f1_score

confusion_mat = confusion_matrix(y_test,predicted)
print(confusion_mat)
from sklearn.metrics import classification_report
target_names = ["sport", "world", "us", "business", "health", "entertainment", "sci_tech"]

print(classification_report(y_test, predicted, target_names=target_names))

[[87  0  0  0  0  0  0]
 [ 0 73  0  0  0  0  0]
 [ 0  0 41  0  0  0  0]
 [ 0  0  0 66  0  0  0]
 [ 0  0  0  0 23  0  0]
 [ 0  0  0  0  0 40  0]
 [ 0  0  0  0  0  0 29]]
               precision    recall  f1-score   support

        sport       1.00      1.00      1.00        87
        world       1.00      1.00      1.00        73
           us       1.00      1.00      1.00        41
     business       1.00      1.00      1.00        66
       health       1.00      1.00      1.00        23
entertainment       1.00      1.00      1.00        40
     sci_tech       1.00      1.00      1.00        29

     accuracy                           1.00       359
    macro avg       1.00      1.00      1.00       359
 weighted avg       1.00      1.00      1.00       359



In [16]:
from sklearn.ensemble import RandomForestClassifier
clf_rdf = RandomForestClassifier()
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.25, random_state=42)
clf_rdf.fit(X_train_tfidf, training_data.flag)
pickle.dump(clf_rdf, open("rdf.pkl", "wb"))

In [17]:
predicted = clf_rdf.predict(X_test)
result_rdf = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_rdf.to_csv('res_rdf.csv', sep = ',')
for predicted_item, result in zip(predicted, y_test):
    print(target_names[predicted_item], ' - ', target_names[result])

world  -  world
world  -  world
business  -  business
world  -  world
business  -  business
sport  -  sport
sport  -  sport
sport  -  sport
us  -  us
sci_tech  -  sci_tech
business  -  business
entertainment  -  entertainment
business  -  business
sport  -  sport
world  -  world
world  -  world
entertainment  -  entertainment
sport  -  sport
world  -  world
us  -  us
world  -  world
world  -  world
health  -  health
sport  -  sport
sci_tech  -  sci_tech
sport  -  sport
entertainment  -  entertainment
business  -  business
business  -  business
business  -  business
world  -  world
sport  -  sport
entertainment  -  entertainment
business  -  business
entertainment  -  entertainment
world  -  world
sport  -  sport
sport  -  sport
us  -  us
sport  -  sport
us  -  us
entertainment  -  entertainment
us  -  us
world  -  world
business  -  business
business  -  business
us  -  us
business  -  business
health  -  health
us  -  us
world  -  world
sport  -  sport
sci_tech  -  sci_tech
business  

In [18]:
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

confusion_mat = confusion_matrix(y_test,predicted)
print(confusion_mat)
from sklearn.metrics import classification_report
target_names = ["sport", "world", "us", "business", "health", "entertainment", "sci_tech"]

print(classification_report(y_test, predicted, target_names=target_names))

[[87  0  0  0  0  0  0]
 [ 0 73  0  0  0  0  0]
 [ 0  0 41  0  0  0  0]
 [ 0  0  0 66  0  0  0]
 [ 0  0  0  0 23  0  0]
 [ 0  0  0  0  0 40  0]
 [ 0  0  0  0  0  0 29]]
               precision    recall  f1-score   support

        sport       1.00      1.00      1.00        87
        world       1.00      1.00      1.00        73
           us       1.00      1.00      1.00        41
     business       1.00      1.00      1.00        66
       health       1.00      1.00      1.00        23
entertainment       1.00      1.00      1.00        40
     sci_tech       1.00      1.00      1.00        29

     accuracy                           1.00       359
    macro avg       1.00      1.00      1.00       359
 weighted avg       1.00      1.00      1.00       359



In [19]:
from sklearn.metrics import classification_report
target_names = ["sport", "world", "us", "business", "health", "entertainment", "sci_tech"]

print(classification_report(y_test, predicted, target_names=target_names))

               precision    recall  f1-score   support

        sport       1.00      1.00      1.00        87
        world       1.00      1.00      1.00        73
           us       1.00      1.00      1.00        41
     business       1.00      1.00      1.00        66
       health       1.00      1.00      1.00        23
entertainment       1.00      1.00      1.00        40
     sci_tech       1.00      1.00      1.00        29

     accuracy                           1.00       359
    macro avg       1.00      1.00      1.00       359
 weighted avg       1.00      1.00      1.00       359



In [20]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

target_names = ["sport", "world", "us", "business", "health", "entertainment", "sci_tech"]



docs_new = input('Enter your input:')
docs_new = [docs_new]
#LOAD MODEL
loaded_vec = CountVectorizer(vocabulary=pickle.load(open("count_vector.pkl", "rb")))
loaded_tfidf = pickle.load(open("tfidf.pkl","rb"))
loaded_model = pickle.load(open("rdf.pkl","rb"))

X_new_counts = loaded_vec.transform(docs_new)
X_new_tfidf = loaded_tfidf.transform(X_new_counts)
import numpy as np
predicted = loaded_model.predict(X_new_tfidf)

print(target_names[predicted[0]])

Enter your input:Messi is going to join a new football club
sport
